### Test 1 for lcode 3d with a single time step and an alternative beam generator.

In [1]:
# Import required modules
from copy import copy

import numpy as np
import time

from lcode2dPy.plasma3d_gpu.initialization import init_plasma
from lcode2dPy.push_solver_3d_gpu import PushAndSolver3d

from lcode2dPy.beam3d_gpu import beam

from lcode2dPy.config.default_config import default_config 

In [ ]:
# Create a config
config = copy(default_config)

config.set('geometry', '3d')
config.set('window-width-step-size', 0.02)
config.set('window-width', 769*0.02)

config.set('window-length', 0.5)
config.set('xi-step', 0.02)

config.set('time-limit', 200.5)
config.set('time-step', 100)

config.set('reflect-padding-steps', 10)
config.set('plasma-padding-steps', 10)
config.set('plasma-fineness', 2)

# Make ready a pusher-solver
pusher_solver = PushAndSolver3d(config)

In [ ]:
# Create a beam
from lcode2dPy.alt_beam_generator.beam_generator import generate_beam
from lcode2dPy.alt_beam_generator.beam_shape import BeamShape
from lcode2dPy.alt_beam_generator.beam_segment_shape import BeamSegmentShape

beam_current = 0.01 * (2*np.pi) # WHY 2*np.pi???
beam_shape = BeamShape(0.01 * 2*np.pi, 2000)

beam_segment = BeamSegmentShape()
beam_shape.add_segment(beam_segment)

In [ ]:
# Load the beam
beam_particles = beam.BeamParticles(0)
beam_particles.init_generated(generate_beam(config, beam_shape))
beam_particles.save('beamfile.npz')
beam_source = beam.BeamSource(config, beam_particles)
beam_drain  = beam.BeamDrain()

In [ ]:
# Simulation loop
start_time = time.time()
time_limit = config.getfloat('time-limit')
time_step_size = config.getfloat('time-step')
time_steps = int(time_limit / time_step_size)


for t_i in range(time_steps):
    pl_fields, pl_particles, pl_currents, pl_const_arrays = init_plasma(config)

    pusher_solver.step_dt(pl_fields, pl_particles, pl_currents, pl_const_arrays,
                          beam_source, beam_drain)

    # print(beam_drain.beam_buffer.size)
    beam_source = beam.BeamSource(config, beam_drain.beam_buffer)
    beam_drain  = beam.BeamDrain()

    t = (t_i + 1) * time_step_size
    print(t)
    # beam_source.beam.save(f'beamfile_{t:+09.2f}_cpu.npz')

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Should work correctly. Compare results with the test 1 result with a rigid beam:
# xi         Ez_00
# xi=+0.0000 +0.0000e+00
# xi=+0.0200 -1.4490e-07
# xi=+0.0400 -7.2443e-07
# xi=+0.0600 -2.0278e-06
# xi=+0.0800 -4.3436e-06
# xi=+0.1000 -7.9592e-06
# xi=+0.1200 -1.3161e-05
# xi=+0.1400 -2.0233e-05
# xi=+0.1600 -2.9458e-05
# xi=+0.1800 -4.1116e-05
# xi=+0.2000 -5.5486e-05
# xi=+0.2200 -7.2842e-05
# xi=+0.2400 -9.3456e-05
# xi=+0.2600 -1.1760e-04
# xi=+0.2800 -1.4553e-04
# xi=+0.3000 -1.7751e-04
# xi=+0.3200 -2.1381e-04
# xi=+0.3400 -2.5467e-04
# xi=+0.3600 -3.0033e-04
# xi=+0.3800 -3.5105e-04
# xi=+0.4000 -4.0706e-04
# xi=+0.4200 -4.6859e-04
# xi=+0.4400 -5.3587e-04
# xi=+0.4600 -6.0912e-04
# xi=+0.4800 -6.8855e-04
# xi=+0.5000 -7.7437e-04